# Cell Tower Optimization Assignment

The code in this notebook is divided into two main sections.  The first contains variations in determining a good, feasible set of towers to (i) use as a starting solution for Gurobi, and (ii) to include in the optimization formulation.

# Import Packages, Get Data

This code uses a stored procedure with two <code>SELECT</code> queries so that both database tables are retrieved with a single stored procedure.

In [9]:
import numpy as np
import gurobipy as gpy
import mysql.connector as mysql
import matplotlib.pyplot as plt

In [10]:
db_user = 'root'
db_pwd = 'MySQL'
db_host='127.0.0.1'
db_db = 'set_cover'

data = []
cnx = mysql.connect(user=db_user, passwd=db_pwd, host=db_host, db=db_db)
cursor = cnx.cursor()
cursor.callproc('spGetData')
for result in cursor.stored_results():
    data.append(result.fetchall())
    
tow = np.array(data[0])
dem = np.array(data[1])
del data

## Compute Distances

This function computes distances between each tower and demand location using <code>numpy</code> in a reasonably fast manner.  The <code>numpy.repeat()</code> and  <code>numpy.tile()</code> methods are used to create the arrays <code>D</code> and <code>T</code> for demand locations and tower locations, respectively, that have outer dimensions equal to the dimensions of the final array that contains the distances between each pair of demand location and tower locations.  This helps to make the element-wise distance computation with <code>numpy</code> fast.

In [11]:
dist_thres = 100

D = np.tile(dem.reshape(len(dem),2),tow.shape[0]).reshape(len(dem),tow.shape[0],2)
T = np.repeat(tow.reshape(1,1,tow.shape[0],2),dem.shape[0], axis=1).reshape(dem.shape[0],tow.shape[0],2)
A = np.sqrt(((D-T)**2).sum(axis=2))
A[A<=dist_thres] = 1
A[A>dist_thres] = 0
A = A.astype(np.int32)
del D
del T
print('Matrix A computed.')

Matrix A computed.


# Finding Good Subsets of Towers and Starting Solutions

## Greedy Algorithm

The concept is to sequentially add towers to a starting solution one at a time by selecting the tower in each iteration that covers the greatest number of remaining demand points that are not yet served by any of the selected towers.  This algorithm is implemented in two ways, both of which create two sets of tower indices:

- <code>col</code>: a starting solution for Gurobi that is feasible in that this tower subset has at least one tower serving each demand location
- <code>col_me2</code>:  a list of other towers that may provide improved solutions, which were identified in the search for a feasible starting solution

The alternative towers in <code>col_me2</code> are included because the greedy solution is not always a great solution.

![cell_tower_greedy_flowchart](cell_flowchart.jpg)

[Youtube_video](https://youtu.be/TP215ZHBozI)

The first approach uses two numpy arrays to keep track of which demand locations are not yet served by the towers selected thus far (<code>idx_dem</code>) and the cell towers that have not yet been selected (<code>idx_tow</code>).

In [12]:
''' Greedily generate minimal coverage of demand locations '''
num_to_add = 20  # number of additional attractive towers to add when adding each tower to the feasible solution

''' Get solutions from file from previous runs if available '''
filename = f'col_{num_to_add}.txt'
filename1 = f'col_me2_{num_to_add}.txt'
try:
    col = np.genfromtxt(filename).astype(np.int32).tolist()
    col_me2 = np.genfromtxt(filename1).astype(np.int32).tolist()
except:
    idx_dem = np.array(range(dem.shape[0]))  # row/demand location indices
    idx_tow = np.array(range(tow.shape[0]))  # column/tower indices
    col = []                                 # empty list for indices of cell tower selected
    col_me2 = []                             # empty list to accumulate ancillary towers
    
    graph = False
    i = 0
    while idx_dem.shape[0] > 0:
        A_sum = A[idx_dem, :][:,idx_tow].sum(axis=0)
        tow_next_idx = np.argmax(A_sum)
        tow_next = idx_tow[tow_next_idx]
        col.append(tow_next)  # select tower that serves greatest number of remaining demand locations
        col_me2.extend(idx_tow[np.argsort(A_sum)[-num_to_add:-1]].tolist()) # Appends other 'best' columns/towers
        idx_dem = np.delete(idx_dem, np.where(A[idx_dem, :][:, idx_tow][:,tow_next_idx]==1))
        idx_tow = np.delete(idx_tow, np.where(idx_tow==tow_next))
        print(f'{len(col)} towers selected; {idx_dem.shape[0]} demand locations not served')
        
        ''' Plot towers and demand '''
        if graph:
            fig,ax = plt.subplots()
            x,y = [*zip(*dem)]
            ax.scatter(x,y, c='k', alpha=0.3, s=1, label='Demand')
            x,y = [*zip(*tow[col])]
            ax.scatter(x,y, c='r', alpha=1.0, s=3, label='Tower')
            ax.scatter(x,y, c='r', alpha=0.2, s=4000)
            ax.set_xlabel('x coordinate')
            ax.set_ylabel('y coordinate')
            plt.savefig(f'movie/cell_{i}.jpg', dpi=600)
            i += 1
            plt.show()
                   
    np.savetxt(filename,col)
    np.savetxt(filename1,col_me2)
    print(f'Greedy start solution has {len(col)} towers; col_me2 has {len(col_me2)} towers.')

This approach uses what is to some, possibly, a more intuitive approach althought the coding is more opaque than in the latter approach to identify which cell tower serves the greatest number of unserved demand locations.

In [ ]:
''' Greedily generate minimal coverage of demand locations '''
num_to_add = 12
filename = f'col_{num_to_add}.txt'
filename1 = f'col_me2_{num_to_add}.txt'
try:
    col = np.genfromtxt(filename).astype(np.int32).tolist()
    col_me2 = np.genfromtxt(filename1).astype(np.int32).tolist()
except:
    col_me2 = []
    I = np.identity(A.shape[1])
    col = [np.argmax(A_sum)]
    dem_cov = A[:,col]
    while dem_cov.all() < 1:
        a = (A@I) - dem_cov.reshape(A.shape[0],-1)
        a[a<0] = 0
        col.append(np.argmax(a.sum(axis=0))) # Appends 'best' column/tower
        col_me2.extend(np.argsort(a.sum(axis=0))[-num_to_add:-1].tolist()) # Appends 5 'best' columns/towers
        dem_cov = A[:,col].max(axis=1)
        print(col[-1], dem_cov.sum())
    np.savetxt(filename,col)
    np.savetxt(filename1,col_me2)
print(f'Greedy start solution has {len(col)} towers.')

# Another Approach

Consider alternate towers.

In [15]:
col = [i for i in range(tow.shape[0]) if i%2==1]
col_me2 = []

print(f'Number of unserved demand locations: {(A[:,col].sum(axis=1) == 0).sum()}')

Number of unserved demand locations: 0


# Gurobi Optimization Model

This Gurobi optimization model considers cell towers identified by 3 criteria.

- The feasible solution identified by the preceding greedy algorithm
- The ancillary towers identified in the search for greedy search for a feasible solution
- An additional number of towers added based on the number of demand locations that they serve

Additional features include:

- Saving a solution file
- Reading a solution file if one exists for the parameters, and using that as a starting solution
  - Otherwise using the greedy feasible solution as a starting solution
- Providing a while loop that adds additional towers in case a solution is infeasible, which shuld not be needed given the feasibiltiy of the starting solution, whichever mode is used to create a starting solution.

This first optimization model is used with the greedy solution as the starting solution.

In [14]:
solved = False
grw = 0.3
A_sum = A.sum(axis=0)
num_tow_sub = 500  # additional towers to add in descending order of the number of demand locations served

while not solved and num_tow_sub < A.shape[1]:
    col_cat = list(set(col + col_me2 + [x for x in np.flip(A_sum.argsort())[:num_tow_sub]]))
    A_sub = A[:,col_cat]
    
    ''' Create model '''
    m = gpy.Model('CellTower')
    m.ModelSense = gpy.GRB.MINIMIZE
    
    ''' Set Gurobi solve parameters '''
    m.Params.TimeLimit = 36*3600 #7200
    m.Params.MIPGap = 0.015
    
    ''' Create dv for towers '''
    x = m.addMVar((A_sub.shape[1],), vtype=gpy.GRB.BINARY, name='y')
    m.update()
    
    ''' Load previous sol if applicable, or otherwise use greedy solution as variable start values '''
    try:
        filesol = f'cell_{num_to_add}_{num_tow_sub}.sol'
        m.read(filesol)
    except:
        x.Start = np.array([1 if c in col else 0 for c in col_cat])
    m.update()
    print(f'{x.Start.sum()} 1s in MIPStart')
    print(f'{(A_sub@x.Start)}')
    
    
    ''' Create objective function '''
    m.setObjective(x.sum(), gpy.GRB.MINIMIZE)  
    
    ''' Add constraints '''
    m.addMConstr(A_sub, x, gpy.GRB.GREATER_EQUAL, np.ones(A_sub.shape[0]), name='Cover')
    m.update()
    
    m.optimize()
    
    if m.Status == 2 or m.Status == 9:
        solved = True
    else:
        num_tow_sub = int((1+grw)*num_tow_sub)
        print(f'num_tow_sub: {num_tow_sub}')
    
m.write(filesol)
print(m.ObjVal)

Set parameter TimeLimit to value 129600
Set parameter MIPGap to value 0.015
42.0 1s in MIPStart
[1. 1. 1. ... 1. 1. 1.]
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-1650 v3 @ 3.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2000 rows, 1147 columns and 71919 nonzeros
Model fingerprint: 0xfeb0a9da
Variable types: 0 continuous, 1147 integer (1147 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded user MIP start with objective 42

Presolve removed 1196 rows and 298 columns
Presolve time: 0.23s
Presolved: 804 rows, 849 columns, 26753 nonzeros
Variable types: 0 continuous, 849 integer (849 binary)

Root relaxation: objective 4.049069e+01, 1394 iterations, 0.07 seconds (0.08 work units)

    Nodes    |    Current Node    |     Obje

This optimization formulation is used with the simple pruning of the cell tower locations.

In [16]:
col_cat = col
A_sub = A[:,col_cat]

''' Create model '''
m = gpy.Model('CellTower')
m.ModelSense = gpy.GRB.MINIMIZE

''' Set Gurobi solve parameters '''
m.Params.TimeLimit = 120
m.Params.MIPGap = 0.10

''' Create dv for towers '''
x = m.addMVar((A_sub.shape[1],), vtype=gpy.GRB.BINARY, name='y')
m.update()

''' Load previous sol if applicable, or otherwise use greedy solution as variable start values '''
filesol = 'cell_alt.sol'
try:
    m.read(filesol)
except:
    x.Start = np.array([1 if c in col else 0 for c in col_cat])
m.update()
print(f'{x.Start.sum()} 1s in MIPStart')
print(f'{(A_sub@x.Start)}')


''' Create objective function '''
m.setObjective(x.sum(), gpy.GRB.MINIMIZE)  

''' Add constraints '''
m.addMConstr(A_sub, x, gpy.GRB.GREATER_EQUAL, np.ones(A_sub.shape[0]), name='Cover')
m.update()

m.optimize()
    
m.write(filesol)
print(m.ObjVal)

Set parameter TimeLimit to value 120
Set parameter MIPGap to value 0.1
41.0 1s in MIPStart
[1. 1. 1. ... 2. 1. 1.]
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-1650 v3 @ 3.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2000 rows, 2244 columns and 127998 nonzeros
Model fingerprint: 0x5c75af14
Variable types: 0 continuous, 2244 integer (2244 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded user MIP start with objective 41

Presolve removed 485 rows and 399 columns
Presolve time: 0.38s
Presolved: 1515 rows, 1845 columns, 92657 nonzeros
Variable types: 0 continuous, 1845 integer (1845 binary)

Root relaxation: objective 3.655343e+01, 5625 iterations, 1.46 seconds (1.82 work units)

    Nodes    |    Current Node    |     Objec

## Make Movies of the Tower Selection Algorithm

First, compile all frames in a numpy array.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from PIL import Image
import numpy as np

root = ['movie/cell_']

for r in root:
    i = 0
    concat = []
    done = False
    while not done:
      try:
          img = np.array(Image.open(f'{r}{i}.jpg'))
          concat.append(img)
          print(f'{r}{i}.jpg', img.shape)
          i += 1
          
      except:
          done = True

    result = np.stack(concat)
    print(r, result.shape)
    np.savetxt(f'{r}shape.txt',np.array(result.shape).astype(np.int16))
    #np.savetxt(f'{r[:-1]}.txt',result.flatten().astype(np.int16))
    result = result.flatten().astype(np.int16)
    np.savez_compressed(f'{r[:-1]}',x=result)

Then, create mp4 files.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from PIL import Image

root = ['movie/cell_']

for r in root:
    m_shape = np.loadtxt(f'{r}shape.txt').astype(np.int32)
    
    size = m_shape[1:]
    fig,ax = plt.subplots()
    #ax.spines['top'].set_visible(False)
    #ax.spines['bottom'].set_visible(False)
    #ax.spines['left'].set_visible(False)
    #ax.spines['right'].set_visible(False)
    #ax.set_xticks([])
    #ax.set_yticks([])
    #ax.margins(0.0)
    ax.axis('off')
    start = np.zeros(size, dtype=np.int16)
    start.fill(255)
    img = ax.imshow(start)
    fps_val = 3
    
    #movie = np.loadtxt(f'{r[:-1]}.txt').reshape(*m_shape).astype(np.int16)
    loaded = np.load(f'{r[:-1]}.npz')
    movie = loaded['x'].reshape(*m_shape).astype(np.int16)
    
    def img_serve(i):
        if i % fps_val == 0:
            print( '.', end ='' )
        img.set_array(np.array(Image.open(f'{r}{i}.jpg')))
        yield [img]
    
    def animate_func(i):
        if i % fps_val == 0:
            print( '.', end ='' )
        #img.axes.set_xticks([])
        #img.axes.set_yticks([])
        #ax.margins(0.0)
        img.set_array(movie[i].reshape(*size)) # the variable movie refers to a numpy array of images
        return [img]


    anim = animation.FuncAnimation(fig, animate_func, frames=m_shape[0])   # , frames = nSeconds * fps, interval = 1000 / fps
    #anim = animation.FuncAnimation(fig, img_serve, frames=100)   # , frames = nSeconds * fps, interval = 1000 / fps
    plt.rcParams['animation.ffmpeg_path'] = r'C:\ProgramData\Anaconda3\envs\base_home\pkgs\ffmpeg-4.3.1-ha925a31_0\Library\bin\ffmpeg.exe'
    Writer = animation.writers['ffmpeg']
    writer = Writer(fps=fps_val, metadata=dict(artist='Me'), bitrate=1800)
    anim.save(f'{r}anim.mp4', writer = writer, dpi=600)